# Dieses Notebook macht die Prognosen für die nächsten 5 Jahre

## Prognose der durchschnittlichen Sonnenstunden mit Konfidenzintervall

In [ ]:
# Bibliotheken importieren
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

# Dateipfade für die CSV-Dateien definieren
file_path_sun_data = '../data/meteo/meteo_swiss_data.csv'
file_path_stn_data = '../data/meteo/meteo_swiss_stn_data.csv'

# Daten laden
sun_data = pd.read_csv(file_path_sun_data, delimiter=';')
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')

# Einträge für Liechtenstein entfernen
stn_data = stn_data[stn_data['canton'] != 'LI']

# Wetter- und Stationsdaten verknüpfen
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Zeitspalte in Datumsformat konvertieren
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.to_period('M').astype(str)

# '-' durch '0' ersetzen und in Float konvertieren
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Ungültige Werte entfernen
data = data[(data['sunhours'] > 0) & (data['year'] != 2025)]

# Monatliche Daten aggregieren
monthly_data = data.groupby(data['time'].dt.to_period('M'))['sunhours'].mean().reset_index()
monthly_data['time'] = pd.to_datetime(monthly_data['time'].astype(str), format='%Y-%m')

# Sicherstellen, dass die Zeitreihe lückenlos ist
full_date_range = pd.date_range(start=monthly_data['time'].min(), end=monthly_data['time'].max(), freq='MS')
df = monthly_data.set_index('time').reindex(full_date_range).rename_axis('time').reset_index()
df['sunhours'] = df['sunhours'].fillna(0)

# SARIMAX-Modell trainieren
model = SARIMAX(df['sunhours'], order=(1,1,1), seasonal_order=(1,1,1,12), enforce_invertibility=False)
results = model.fit()

# Prognose für 72 Monate (6 Jahre) erstellen
forecast = results.get_forecast(steps=72)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Interaktives Diagramm mit Plotly erstellen
fig = go.Figure()

# Historische Daten hinzufügen
fig.add_trace(go.Scatter(x=df['time'], y=df['sunhours'], mode='lines', name='Historische Daten', line=dict(color='#1f77b4')))

# Prognosedaten hinzufügen
forecast_index = pd.date_range(start=df['time'].iloc[-1], periods=72, freq='MS')
fig.add_trace(go.Scatter(x=forecast_index, y=forecast_mean, mode='lines', name='Prognose', line=dict(color='#ff7f0e')))

# Konfidenzintervall als schattierten Bereich hinzufügen
fig.add_trace(go.Scatter(
    x=list(forecast_index) + list(forecast_index[::-1]),
    y=list(forecast_ci.iloc[:, 1]) + list(forecast_ci.iloc[:, 0][::-1]),
    fill='toself',
    fillcolor='rgba(255, 127, 14, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    name='Konfidenzintervall'
))

# Layout anpassen
fig.update_layout(
    title={
        'text': 'Prognose der durchschnittlichen Sonnenstunden mit Konfidenzintervall',
        'x': 0.5,  # Zentriere den Titel
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial'}
    },
    yaxis_title='Durchschnittliche Sonnenstunden',
    template='plotly_white',
    legend=dict(
        font=dict(size=12),
        orientation='h',
        x=0.5,
        xanchor='center',
        y=-0.2
    ),
    xaxis=dict(
        tickformat='%Y',
        dtick='M12',
        ticklabelmode='period',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Werte im Hover ohne Kommastellen anzeigen
fig.update_traces(hovertemplate='%{x|%Y-%m}: %{y:.0f}')

# Diagramm anzeigen
fig.show()

# Diagramm als HTML-Datei speichern
fig.write_html('../docs/assets/diagramme/sunhours_forecast.html')

## Polynomiale Prognose (Grad 2) der kumulierten Leistung je Kanton

In [ ]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# CSV-Datei mit den PV-Anlagen laden
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Subkategorien laden und zuordnen
sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))

# Nur Photovoltaik-Anlagen filtern
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr aus 'BeginningOfOperation' extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Daten gruppieren und kumulativ berechnen
cumulative_data = df.groupby(['Year', 'Canton'])['TotalPower'].sum().groupby(level=1).cumsum().reset_index()
filtered_data = cumulative_data[(cumulative_data['Year'] >= 2015) & (cumulative_data['Year'] <= 2024)].copy()
filtered_data['CumulativePower'] = cumulative_data.groupby('Canton')['TotalPower'].cumsum()

# Top 4 Kantone mit höchstem Endwert filtern
end_values = filtered_data.groupby('Canton')['CumulativePower'].max()
top_4_cantons = end_values.nlargest(4).index.tolist()
df = filtered_data[filtered_data['Canton'].isin(top_4_cantons)].drop(columns=['TotalPower']).copy()

# Prognosejahre definieren
years_forecast = np.arange(2024, 2031)

# Neues Diagramm erstellen
fig = go.Figure()

# Farben für die Kantone definieren
colors = {
    'BE': '#1f77b4',  # Blau für Bern
    'ZH': '#ff7f0e',  # Orange für Zürich
    'VD': '#2ca02c',  # Grün für Waadt
    'SG': '#d62728'   # Rot für St. Gallen
}

# Daten für jeden Kanton verarbeiten
for canton in df['Canton'].unique():
    df_canton = df[df['Canton'] == canton].copy()
    x = df_canton['Year'].values
    y = df_canton['CumulativePower'].values

    # Historische Werte plotten
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers+lines',
        name=f'{canton} (historisch)',
        line=dict(color=colors[canton])
    ))

    try:
        # Polynom 2. Ordnung fitten
        coeffs = np.polyfit(x, y, deg=2)
        poly_func = np.poly1d(coeffs)
        y_forecast = poly_func(years_forecast)

        # Prognose plotten
        fig.add_trace(go.Scatter(
            x=years_forecast, y=y_forecast,
            mode='lines',
            line=dict(dash='dot', color=colors[canton]),
            name=f'{canton} (Prognose)'
        ))
    except Exception as e:
        print(f"{canton}: Fehler beim Fit: {e}")

# Layout des Diagramms anpassen
fig.update_layout(
    title={
        'text': 'Polynomiale Prognose der kumulierten Leistung je Kanton',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial'}
    },
    yaxis_title='Kumulative Leistung (kWp)',
    hovermode='x unified',
    template='plotly_white',
    legend=dict(
        font=dict(size=12),
        orientation='h',
        x=0.5,
        xanchor='center',
        y=-0.2
    ),
    xaxis=dict(
        tickformat='%Y',
        dtick=1,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Werte im Hover ohne Kommastellen und in Millionen anzeigen
fig.update_traces(hovertemplate='%{x}: %{y:.1s}')

# Diagramm anzeigen
fig.show()

# Diagramm als HTML-Datei speichern
fig.write_html('../docs/assets/diagramme/cumulative_power_forecast.html')

## Kumulative Prognose der gesamten Schweiz

In [ ]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# CSV-Datei mit den PV-Anlagen laden
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Subkategorien laden und zuordnen
sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))

# Nur Photovoltaik-Anlagen filtern
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr aus 'BeginningOfOperation' extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Daten gruppieren und kumulativ berechnen
cumulative_data = df.groupby(['Year'])['TotalPower'].sum().reset_index()
cumulative_data['CumulativePower'] = cumulative_data['TotalPower'].cumsum()
filtered_data = cumulative_data[(cumulative_data['Year'] >= 2015) & (cumulative_data['Year'] <= 2024)].copy()

# Historische Werte
x = filtered_data['Year'].values
y = filtered_data['CumulativePower'].values

# Prognosejahre definieren
years_forecast = np.arange(2024, 2031)

# Polynom 2. Ordnung fitten
coeffs_switzerland = np.polyfit(x, y, deg=2)
poly_func_switzerland = np.poly1d(coeffs_switzerland)
y_forecast_switzerland = poly_func_switzerland(years_forecast)

# Diagramm erstellen
fig_switzerland = go.Figure()

# Historische Werte plotten
fig_switzerland.add_trace(go.Scatter(
    x=x, y=y,
    mode='markers+lines',
    name='Schweiz (historisch)',
    line=dict(color='#1f77b4')
))

# Prognose plotten
fig_switzerland.add_trace(go.Scatter(
    x=years_forecast, y=y_forecast_switzerland,
    mode='markers+lines',
    line=dict(dash='dot', color='#ff7f0e'),
    name='Schweiz (Prognose)'
))

# Layout des Diagramms anpassen
fig_switzerland.update_layout(
    title={
        'text': 'Kumulative Prognose der gesamten Schweiz',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial'}
    },
    yaxis_title='Kumulative Leistung [kWp]',
    hovermode='x unified',
    template='plotly_white',
    legend=dict(
        font=dict(size=12),
        orientation='h',
        x=0.5,
        xanchor='center',
        y=-0.2
    ),
    xaxis=dict(
        tickformat='%Y',
        dtick=1,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Werte im Hover ohne Kommastellen anzeigen
fig_switzerland.update_traces(hovertemplate='%{x}: %{y:.0s}')

# Diagramm anzeigen
fig_switzerland.show()

# Diagramm als HTML-Datei speichern
fig_switzerland.write_html('../docs/assets/diagramme/cumulative_power_forecast_switzerland.html')

## Durchschnittliche PV-Vergütung pro Jahr in der Schweiz (inkl. Prognose)

In [ ]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.optimize import curve_fit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.express as px

# CSV-Dateien laden
evu_df = pd.read_csv('../data/vese/evu_ergebnisse.csv', sep=';')  # EVU-Daten
gemeinde_df = pd.read_csv('../data/vese/gemeinde_ergebnisse.csv', sep=';')  # Gemeinde-Daten

# Datenframes basierend auf `nrElcom` zusammenführen
merged_df = pd.merge(evu_df, gemeinde_df, on='nrElcom', how='inner')  # Inner Join

# Gemeindestand.csv laden, um Kantone und BFS Gde-nummer zuzuordnen
gemeindestand_df = pd.read_csv('../data/vese/Gemeindestand.csv', sep=';')  # Mapping-Daten

# Mapping-Dictionary für Kantone und BFS Gde-nummer erstellen
gemeinde_to_kanton = gemeindestand_df.set_index('BFS Gde-nummer')['Kanton'].to_dict()

# Kantone und BFS Gde-nummer zuordnen
merged_df['Kanton'] = merged_df['idofs'].map(gemeinde_to_kanton)

# Fehlende Werte in 'energy1' und 'eco1' mit 0 auffüllen
merged_df = merged_df.copy()  # Sicherstellen, dass auf einer Kopie gearbeitet wird
merged_df['energy1'] = merged_df['energy1'].fillna(0)  # NaN durch 0 ersetzen
merged_df['eco1'] = merged_df['eco1'].fillna(0)

# Zeilen entfernen, in denen sowohl 'energy1' als auch 'eco1' 0 sind
merged_df = merged_df[~((merged_df['energy1'] == 0) & (merged_df['eco1'] == 0))]

# Vergütung berechnen
merged_df['remuneration'] = merged_df['energy1'] + merged_df['eco1']  # Gesamtvergütung berechnen

df = merged_df.groupby(['year'])['remuneration'].mean().reset_index()  # Durchschnitt berechnen
df.rename(columns={'remuneration': 'avg_remuneration'}, inplace=True)  # Spalte umbenennen

# Durchschnittliche Vergütung auf 2 Dezimalstellen runden
df['avg_remuneration'] = df['avg_remuneration'].round(2)

# Historische Werte
x = df['year'].values
y = df['avg_remuneration'].values

# ETS-Modell (Exponential Smoothing) anpassen
ets_model = ExponentialSmoothing(y, trend="add", seasonal=None, damped_trend=False)
ets_fit = ets_model.fit()

# Prognosewerte berechnen
years_forecast = np.arange(df['year'].max(), 2031)
forecasted_values = ets_fit.forecast(len(years_forecast))

# Diagramm erstellen
fig = go.Figure()

# Historische Werte plotten
fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='markers+lines',
    name='Historisch',
    line=dict(color='#1f77b4')
))

# Prognose plotten
fig.add_trace(go.Scatter(
    x=years_forecast, y=forecasted_values,
    mode='markers+lines',
    line=dict(dash='dot', color='#ff7f0e'),
    name='Prognose'
))

# Layout des Diagramms anpassen
fig.update_layout(
    title={
        'text': 'Durchschnittliche PV-Vergütung pro Jahr in der Schweiz (inkl. Prognose)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial'}
    },
    yaxis_title='Durchschnittliche Vergütung (Rp/kWh)',
    hovermode='x unified',
    template='plotly_white',
    legend=dict(
        font=dict(size=12),
        orientation='h',
        x=0.5,
        xanchor='center',
        y=-0.2
    ),
    xaxis=dict(
        tickformat='%Y',
        dtick=1,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        range=[df['avg_remuneration'].min() - 5, forecasted_values.max() + 2]  # Zoom anpassen
    )
)




# Werte im Hover ohne Kommastellen anzeigen
fig.update_traces(hovertemplate='%{x}: %{y:.0s} Rp/kWh')

# Diagramm anzeigen
fig.show()

# Diagramm als HTML-Datei speichern
fig.write_html('../docs/assets/diagramme/avg_remuneration_forecast.html')